# Scraping the UCSD Capes Website

### Setup

Import necessary libraries.

In [ ]:
import csv
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time 

Open a chrome window and go to the UCSD Cape website.

In [ ]:
chrome_options = Options()
chrome_options.add_argument("--disable-extensions")
driver = webdriver.Chrome(executable_path='/Users/ericnguyen/Downloads/chromedriver')
driver.get('http://cape.ucsd.edu/responses/Results.aspx')

Log in.

In [ ]:
# input username and password
username = driver.find_element_by_id("ssousername")
username.send_keys('enguyen')
password = driver.find_element_by_id("ssopassword")
password.send_keys('you thought')
login = driver.find_element_by_xpath("/html/body/main/div/section/div/div/div[1]/div/div[2]/form/button")
login.click()

driver.implicitly_wait(10)                      # wait for duo auth

### Collecting Data from Tables

In [ ]:
# go through each department in the select box
select_box = Select(driver.find_element_by_xpath("/html/body/div[6]/div/div[2]/div/form/div[3]/div[1]/div[3]/div[2]/select"))
options = select_box.options

# making lists to put in csv
fields = ["Name", "Department", "Rcmd Instr", "Avg Grade"]

rows = []
rowlist = list()

for ele_index in range(1,len(options)):
    select = driver.find_element_by_xpath("/html/body/div[6]/div/div[2]/div/form/div[3]/div[1]/div[3]/div[2]/select")
    select.click()
    select_box.select_by_index(ele_index)
    submit = driver.find_element_by_id("ctl00_ContentPlaceHolder1_btnSubmit")
    submit.click()
    

    time.sleep(30)
  
    WebDriverWait(driver,30).until( EC.presence_of_element_located((By.TAG_NAME, "table")))
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    driver.implicitly_wait(10)
    table = soup.find_all('table')[0]
    rows = table.find_all('tr')
    for tr in rows:
        td = tr.find_all('td')
        row = [i.text for i in td]
        if (row.count('FA18') == 1):
            rowlist.append(row)
        elif (row.count('S218') == 1):
            rowlist.append(row)
        elif (row.count('S118') == 1):
            rowlist.append(row)
        elif(row.count('SP18') == 1):
            rowlist.append(row)
        elif (row.count('WI18') == 1):
            rowlist.append(row)
        

### Output a CSV

In [ ]:
df_bs = pd.DataFrame(rowlist, columns=['Instructor', 'Course', 'Term', 'Enroll', 'Evals Made', 'Rcmd Class', 'Rcmnd Instr', 'Study Hours/week', 'avg Grade Expected', 'Average grade received'])
df_bs.set_index('Instructor', inplace=True)
df_bs.to_csv('beautifulsoup.csv')